#Assessment for Generative AI for Image Generation

###Date:
###Name:
###NRIC (Last 3 digits and alphabet):

###*Submit the completed notebook to the instructor.*

## Task: Complte the below codes to integrate prompt generation (OpenAI API) with image generation (Stability API)

## Instruction
- Generate Prompt using OpenAI Chat API
    - Upload prompt_gen.txt and negtive_prompt.txt
    - read system_message from prompt_gen.txt
    - read negative_prompt from negative_prompt.txt
    - request user_message using input("Your request:")
    - call OpenAI api to generate prompt with Temperature=0.5 and max_tokens=150
- Generate Image using SD Core model with below parameters
    - aspect_ratio = "3:2"
    - seed = 0
    - output_format = "png"
- At the end, your program should be able to generate a high-quality image by requesting a few key words from end user

In [ ]:
!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key  = os.environ["OPENAI_API_KEY"]
sd_api_key = os.environ["STABILITY_API_KEY"]

In [ ]:
!pip install openai

In [7]:
import openai
client = openai.OpenAI()

In [11]:
import IPython
import json
from google.colab import output
import requests
from PIL import Image

In [8]:
# Define functions to call API service
def send_generation_request(host, params,):
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {sd_api_key}"
    }

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response

In [ ]:
# Generate prompt
f = open("instruct_prompt.txt", "r")
system_message = f.read()
f.close()

f = open("negtive_prompt.txt", "r")
negative_prompt = f.read()
f.close()

user_message = input("Your request (key words):")  #"playground, dog, soccer, trees, sunset"

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": _________},
        {"role": "user", "content": _________}
    ],
    temperature= ___,   #0:precise  1:createive
    max_tokens=150,
)

prompt = response.choices[0].message.content
print(prompt)


# Generate Image using SD Core model
aspect_ratio = ____
seed = 0
output_format = "png"

host = f"https://api.stability.ai/v2beta/stable-image/generate/core"

params = {
    "prompt" : _______,
    "negative_prompt" : ___________,
    "aspect_ratio" : ________+,
    "seed" : seed,
    "output_format": output_format,
    "mode" : "text-to-image"
}

response = send_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")

# Save and display result
generated = f"generated_{seed}.{output_format}"
with open(generated, "wb") as f:
    f.write(output_image)
print(f"Saved image {generated}")

# Display Image
output.no_vertical_scroll()
print("Result image:")
IPython.display.display(Image.open(generated))